In [ ]:
!pip install contractions


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub('<.*?>', '', text)  # remove HTML
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # remove links
    text = re.sub(r"[^a-zA-Z\s]", '', text)  # remove punctuation/numbers
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text.strip()

PYTHON= 3.11 , TF =2.12

In [ ]:
# create a dataframe
df = pd.read_csv('/content/drive/MyDrive/colab,zip/IMDB Dataset.csv')
df['review'] = df['review'].astype(str).apply(clean_text)

In [ ]:
print(df)

                                                  review sentiment
0      one of the other reviewers has mentioned that ...  positive
1      a wonderful little production the filming tech...  positive
2      i thought this was a wonderful way to spend ti...  positive
3      basically theres a family where a little boy j...  negative
4      petter matteis love in the time of money is a ...  positive
...                                                  ...       ...
49995  i thought this movie did a down right good job...  positive
49996  bad plot bad dialogue bad acting idiotic direc...  negative
49997  i am a catholic taught in parochial elementary...  negative
49998  im going to have to disagree with the previous...  negative
49999  no one expects the star trek movies to be high...  negative

[50000 rows x 2 columns]


In [ ]:
#Preprocess text
texts = df['review'].astype(str).values
labels = df['sentiment'].values

In [ ]:
# Label encoding
le = LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
# Tokenize
vacab_size = 10000
max_len = 500
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vacab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences,padding='pre', maxlen = max_len, truncating='post')

In [ ]:
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [ ]:
# Split data
# Use the padded sequences (features) and the labels (target)
x_train, x_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, random_state=42)

In [ ]:
# Model
model = Sequential([
    Embedding(vacab_size,64,input_length=max_len),
    GRU(64),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



model.fit(x_train, y_train, epochs=4, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/4
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - accuracy: 0.7085 - loss: 0.5416 - val_accuracy: 0.8826 - val_loss: 0.2863
Epoch 2/4
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 37s 19ms/step - accuracy: 0.9102 - loss: 0.2354 - val_accuracy: 0.8994 - val_loss: 0.2527
Epoch 3/4
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 40s 19ms/step - accuracy: 0.9427 - loss: 0.1618 - val_accuracy: 0.9015 - val_loss: 0.2515
Epoch 4/4
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 40s 18ms/step - accuracy: 0.9657 - loss: 0.1083 - val_accuracy: 0.8960 - val_loss: 0.3014


In [ ]:
model.save("sentiment_model.keras")


In [ ]:
from google.colab import files
files.download("sentiment_model.keras")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

files.download("tokenizer.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>